In [7]:
!pip3 install torch
!pip3 install skorch
from skorch import NeuralNetClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time
import pathlib
import dill
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
preprocessed_folder = pathlib.Path('/content/drive/My Drive/validated_clips/preprocessed/')
mfccs_padded = {"dev":[], "train":[], "test":[]}
labels={"dev":[], "train":[], "test":[]}
for i in range(1,209):
  number = '{:.6f}'.format(i)
  mfcc_filename = number+"mfcc_5accents.pkl"
  label_filename = number+"label_5accents.pkl"
  mfcc_filepath = preprocessed_folder / mfcc_filename
  label_filepath = preprocessed_folder / label_filename

  with open(mfcc_filepath, "rb") as f:
    mfcc_local = dill.load(f)
  mfccs_padded['dev'].extend(mfcc_local['dev'])
  mfccs_padded['train'].extend(mfcc_local['train'])
  mfccs_padded['test'].extend(mfcc_local['test'])
  #print([len(value) for value in mfccs_padded.values()])
  #print([len(value) for value in mfcc_local.values()])
  
  with open(label_filepath, "rb") as f:
    label_local = dill.load(f)
  labels['dev'].extend(label_local['dev'])
  labels['train'].extend(label_local['train'])
  labels['test'].extend(label_local['test'])

print([len(value) for value in mfccs_padded.values()])
print([len(value) for value in labels.values()])

[1759, 12465, 1159]
[1759, 12465, 1159]


In [4]:
print(type(mfccs_padded))
print([len(value) for value in mfccs_padded.values()])
#print(type(mfccs_padded["train"]))
#print(type(mfccs_padded["train"][0]))
#print(mfccs_padded["train"][0])
#print(labels["train"][0])
print([len(array) for array in mfccs_padded["train"]])

<class 'dict'>
[1759, 12465, 1159]
[6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234, 6234

In [5]:
mfccs_padded = {key:np.stack(value) for key,value in mfccs_padded.items()}
print(mfccs_padded['train'].shape)

(12465, 6234, 13)


In [0]:
class downwardSlope(nn.Module):
    def __init__(self, maxSeqLength, outsize):
        super().__init__()
        convOutLength = maxSeqLength
        padding = 1
        dilation = 1
        kernel_size = 15
        stride = 1

        self.conv1 = nn.Conv1d(13, 32, kernel_size=kernel_size, dilation =dilation ,stride=stride, padding=padding)
        #convOutLength calculated using equation in nn.conv1d documentation
        convOutLength = (convOutLength + 2*padding -dilation*(kernel_size-1) -1)//stride + 1
        self.conv2 = nn.Conv1d(32, 32, 15, padding=padding)
        convOutLength = (convOutLength + 2*padding -dilation*(kernel_size-1) -1)//stride + 1
        self.conv3 = nn.Conv1d(32, 32, 15, padding=padding)
        convOutLength = (convOutLength + 2*padding -dilation*(kernel_size-1) -1)//stride + 1

        self.fc1 = nn.Linear(convOutLength, outsize)

    def forward(self,x):
        print("insideforward",type(x), x.dtype)
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return torch.squeeze(x)

In [0]:
def evaluateModel(model, lossCriterion, X, y, batchSize):
    with torch.no_grad():
        n=batchSize
        batched = list(zip([X[i:i + n] for i in range(0, len(X), n)],
                          [y[i:i + n] for i in range(0, len(y), n)]))
        predList = []
        yList = []
        lossMeans = []
        for i,(batchX, batchY) in enumerate(batched):
            print('x',type(X), X.dtype)
            print('batchx',type(batchX), batchX.dtype)
            predProb = model(batchX)
            if np.isnan(predProb.cpu().detach().numpy()).any():
                print("AAAAAA a NAN")
                return
            loss = lossCriterion(predProb, batchY)
            batchPred = predProb.round().tolist()
            predList = predList+batchPred
            batchYargmaxed = batchY.tolist()
            yList = yList+batchYargmaxed
            lossMeans.append(torch.mean(loss).item())

        #acc = accuracy_score(yList, predList)
        #report = classification_report(yList, predList)
        #confMat = confusion_matrix(yList, predList)
        lossMean = sum(lossMeans)/float(len(lossMeans))

    return lossMean

In [0]:
def trainModel(model, optimizer, criterion, train_X, train_y, val_X, val_y, batchSize, startEpoch, endEpoch):
    notifyEvery = 100 if torch.cuda.is_available() else 2

    checkmarkTime = time.time()

    n=batchSize
    batched = list(zip([train_X[i:i + n] for i in range(0, len(train_X), n)],
                      [train_y[i:i + n] for i in range(0, len(train_y), n)]))
    numBatches = len(batched)
    print("number of batches", numBatches)
    for epoch in range(startEpoch, endEpoch):
        print("epoch:", epoch)
        for i,(batchX,batchy) in enumerate(batched):
            optimizer.zero_grad()
            output =  model(batchX)
            loss = criterion(output, batchy)
            loss.backward()
            optimizer.step()
            if np.isnan(output.cpu().detach().numpy()).any():
                print("AAAAAA a NAN")
                return
            if i%notifyEvery ==notifyEvery-1:
                print('[%d, %5d]' %
                  (epoch + 1, i + 1))
                timeTook = time.time() - checkmarkTime
                print("took", timeTook, "seconds for", notifyEvery, "batches")
                if(torch.cuda.is_available()):
                    print(torch.cuda.max_memory_allocated()/1e9, "GB of VRAM being used")
                checkmarkTime = time.time()
        trainLoss = evaluateModel(model,criterion,train_X, train_y, batchSize)
        valLoss = evaluateModel(model, criterion,val_X, val_y, batchSize)
        print('loss', valLoss)
    print("finished training")

In [0]:
train_X = torch.as_tensor(mfccs_padded['train'], dtype=torch.float)
val_X = torch.as_tensor(mfccs_padded['dev'], dtype=torch.float)
print(train_X.shape)
train_X = train_X.permute(0,2,1)
val_X = val_X.permute(0,2,1)

le = preprocessing.LabelEncoder()
enc =  preprocessing.OneHotEncoder(handle_unknown='ignore')
le.fit(labels['dev'])

transformed_val = le.transform(labels['dev'])
onehotted_val =enc.fit_transform(transformed_val.reshape(-1,1)).toarray() 
val_y = torch.as_tensor(onehotted_val, dtype=torch.long)

transformed_train = le.transform(labels['train'])
onehotted_train =enc.fit_transform(transformed_train.reshape(-1,1)).toarray()
train_y = torch.as_tensor(onehotted_train, dtype=torch.long)

seqLength = train_X.shape[1]
outsize = len(le.classes_)
print( seqLength, outsize)
mod = downwardSlope(seqLength, outsize)

device = "cpu"
criterion = nn.CrossEntropyLoss()
endEpoch = 3
learningRate = 0.001
optimizer = torch.optim.Adam(mod.parameters(), lr = learningRate)
print('train_x',type(train_X), train_X.dtype)
trainModel(mod, optimizer,criterion, train_X, train_y, val_X, val_y, 10, 0, 10)